In [144]:

from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import mysql.connector
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, KFold, cross_val_predict, \
    cross_validate
import xgboost as xgb
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


with open('pss.txt') as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
password = lines[0]
db = lines[1]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password=password,
    database=db,
    use_pure=True
)

my_cursor = mydb.cursor()
steps_title_list = []
d_steps_title = {}
university_name_list = []
d_uni_name = {}



my_cursor.execute('select distinct steps_title from useful_data')
for item in my_cursor:
    steps_title_list.append(item)


for title in steps_title_list:
    if title[0] == 'نیازمند تعیین وضعیت':
        d_steps_title[title[0]] = 0
    elif title[0] == 'تایید برای مصاحبه':
        d_steps_title[title[0]] = 1
    elif title[0] == 'استخدام شده':
        d_steps_title[title[0]] = 2
    elif title[0] == 'رد شده':
        d_steps_title[title[0]] = 3
    elif title[0] == 'انصراف از مصاحبه':
        d_steps_title[title[0]] = 4
    else:
        d_steps_title[title[0]] = 6


df_train = pd.read_csv("train.csv")
df_train.head(10)
df_test = pd.read_csv("test.csv")
df_train['steps_title'] = df_train['steps_title'].map(d_steps_title)
df_test['steps_title'] = df_test['steps_title'].map(d_steps_title)
# convert_nans(df_train)
# convert_nans(df_test)
# [2 , 4 , 5 , 6 , 7 ,10]
x_train =df_train[df_train.columns[1:12]]
x_test = df_test[df_test.columns[1:12]]
y_test = df_test['steps_title']
y_train = df_train['steps_title']
kfold = KFold(n_splits=10)
features_train = df_train.columns[1:12]
# clf = RandomForestClassifier(n_jobs=2 , random_state=0)
clf = xgb.XGBClassifier(seed=42, subsample=0.9)
cv_results = cross_val_score(clf,x_train , y_train,cv=kfold , scoring='accuracy')
output = cross_validate(clf, x_train, y_train, cv=10, scoring = 'accuracy', return_estimator =True)
y_pred = cross_val_predict(clf,x_test,y_test,cv=10)
proba = cross_val_predict(clf,x_test,y_test,cv=10, method='predict_proba')
clf.fit(x_train , y_train , verbose=0, eval_set =[(x_test, y_test)])
# clf.fit(x_train , y_train)
results = (cv_results.mean(),cv_results.std())
print(results)
print("accuracy : ", accuracy_score(y_test, y_pred))
print(pd.crosstab(y_test,y_pred))
df_test["prediction"] = y_pred
# for idx,estimator in enumerate(output['estimator']):
#     print("Features sorted by their score for estimator {}:".format(idx))
#     feature_importances = pd.DataFrame(estimator.feature_importances_,
#                                         index= features_train,
#                                         columns=['importance']).sort_values('importance', ascending=False)
#     print(feature_importances)


proba_0 = []
proba_1 = []
type(proba)
for prob0 , prob1 in proba:
    proba_0.append(float(prob0))
    proba_1.append(float(prob1))
df_test['Prob_0'] = proba_0
df_test['Prob_1'] = proba_1
final_df = df_test.sort_values(by=['Prob_0'], ascending=False)
print(final_df.loc[final_df['steps_title'] == final_df['prediction'] , 'job_applicant_id'].iloc[0:9])
# print(final_df.loc['job_applicant_id'].iloc[0:9])
#
final_df.head(50)

/Users/narges/PycharmProjects/sanjemanTest/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/narges/PycharmProjects/sanjemanTest/venv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/narges/PycharmProjects/sanjemanTest/venv/lib/python3.8/site-packages/xgboost/core.py", line 436, in inner_f
    missing=None,
  File "/Users/narges/PycharmProjects/sanjemanTest/venv/lib/python3.8/site-packages/xgboost/sklearn.py", line 1158, in fit
  File "/Users/narges/PycharmProjects/sanjemanTest/venv/lib/python3.8/site-packages/xgboost/sklearn.py", line 236, in _wrap_evaluation_matrices
    self.colsample_bynode = colsample_bynode
  File "/Users/narges/PycharmProjects/sanjemanTest/venv/lib

ValueError: DataFrame.dtypes for data must be int, float, bool or categorical.  When
                categorical type is supplied, DMatrix parameter
                `enable_categorical` must be set to `True`.university